# Ranking Evluation
Mit diesem Notebook evaluieren wir unser Regressionsmodell.

### Vorbereitung
Wir teilen eine aktuelle Regressionamtrix mit stratified sampling in Trainingsdaten (2/3) und Testdaten (1/3). 
Anschliessend lernen wir das Modell an und berechnen die Prediction für die Testdaten. Den Prediction Vektor sortieren wir nach Anzahl Verwundbarkeiten und vergleichen jeden Eintrag mit dem tatsächlichen Wert im Target Vektor der Testdaten.

### Ranking
Um die Qualität der Regression zu evaluieren vergleichen wir nicht die Anzahl Verwundbarkeiten, sondern erstellen zuerst einzeln ein Ranking für die Prediction und den tatsächlichen Target Vektor. Da die Komponenten nach der ANzahl vorhergesagten Verwundbarkeiten sortiert sind, ist dort das ranking ebenfalls sortiert. Gleiche Einträge haben den selben Rang.
<img src="img/ranking.png" alt="Drawing" style="width: 350px"/>

### Evaluation
Wir nehmen nur das oberste 1% der Daten, also jene Komponenten mit besonder viel vorhergesagten Verwundbarkeiten.

#### Spearman
Zuerst berechnen wir die Korrelation anhand des spearman rank corellation coefficient. Mit einem Wert zwischen -1 (negative Korrelation) und +1 (Korrelation) gibt er an, wie gut das Ranking der vorhergesagten und tatsächlichen Verwundbarkeiten im Test Vektor korrelieren. 

Das Problem bei der Korrelation ist, dass der tatsächliche Rang nicht berücksichtigt wird, sondern nur die Ähnlichkeit der Verteilung. Folgende Rankings hätten eine Korrelation von +1, obwohl es im Target Vektor 10 Komponenten gibt, die verwundbarer sind.
<img src="img/ranking_bad.png" alt="Drawing" style="width: 350px"/>

#### Avg Difference
Um den Tatsächlichen Ranking Wert zu berücksichtigen, berechnen wir für jede Komponenten die Differenz zwischen dem Ranking-Wert der Prediction und dem Ranking-Wert des Target Vektors. Von allen Komponenten des Top 1% berechnen wir anshcliessend den Durchschnitt.

In [11]:
from imports.matrix_helper import MatrixHelper
from imports.prediction_helper import PredictionHelper
from scipy.stats import rankdata
from scipy.stats import spearmanr
import numpy as np

matrix_helper = MatrixHelper()
matrices = matrix_helper.load_from_parse('data/matrices/matrix_reg_incl_history.pickle')

# Instantiate Prediction Helper Class and predict values for compare matrix
prediction_helper = PredictionHelper()
prediction_helper.calculate_validation_compare_matrix(matrices, sampling_factor=(2.0/3), prediction_type='SVR', crop_matrix=False)
compare_matrix = prediction_helper.get_compare_matrix_sorted()

# Create ranking 
predicted_ranking = len(compare_matrix[:, 1]) - rankdata(np.array(compare_matrix[:,1], dtype='f'), method='max') + 1
actual_ranking = len(compare_matrix[:, 2]) - rankdata(np.array(compare_matrix[:,2], dtype='f'), method='max') + 1

# Crop first 1% sorted by predicted ranks
relevant_samples_count = int(round(0.01 * len(predicted_ranking)))
predicted_ranking = predicted_ranking[range(relevant_samples_count)]
actual_ranking = actual_ranking[range(relevant_samples_count)]

# Calc Spearman's correlation coefficient
spearman = spearmanr(compare_matrix[range(relevant_samples_count), 1], compare_matrix[range(relevant_samples_count), 2])

# Calc difference between predicted and actual ranking and avg diff
diff = abs(np.subtract(actual_ranking, predicted_ranking))
avg_diff = float(sum(diff))/relevant_samples_count

# Print
print('Spearman correlation coefficient: {}'.format(spearman[0]))
print('Average difference: {}'.format(avg_diff))
print

print('{:25}  {:>3} {:>3} '.format('Component', 'Act.', 'Pred.'))
for i in range(relevant_samples_count):
#    print('{},,{},{},,{},{}'.format(
#        compare_matrix[i][0], compare_matrix[i][2], actual_ranking[i], compare_matrix[i][1], predicted_ranking[i]))
    print('{:25} |{:>3} - {:>3}| = {:>3}'.format(
        compare_matrix[i][0], actual_ranking[i], predicted_ranking[i], diff[i]))



Spearman correlation coefficient: 0.9150512824
Average difference: 7.96825396825

Component                  Act. Pred. 
nsDocument                |  2 -   1| =   1
nsDocument                |  1 -   1| =   0
nsDocument                |  3 -   3| =   0
nsDocument                |  4 -   3| =   1
nsDocument                |  5 -   5| =   0
nsDocShell                |  7 -   6| =   1
nsDocShell                |  8 -   7| =   1
nsDocument                |  8 -   8| =   0
nsDocument                | 10 -   9| =   1
nsDocShell                | 13 -  10| =   3
jsobj                     |  6 -  10| =   4
nsGlobalWindow            | 13 -  12| =   1
jsobj                     | 15 -  12| =   3
nsDocShell                | 15 -  14| =   1
nsGlobalWindow            | 18 -  14| =   4
jsobj                     | 12 -  14| =   2
nsContentUtils            | 18 -  17| =   1
nsCSSFrameConstructor     | 18 -  18| =   0
nsGlobalWindow            | 26 -  19| =   7
jsobj                     | 18 -  19| =   1